In [ ]:
# # ESE 5390 Lab 4

# This lab will walk through an example of pruning a model to reduce the size of the network while mitigating accuracy loss.
# The benefits of pruning include lower storage costs and possibly lower computation cost for optimized architectures.

# This lab makes heavy use of prior labs and tutorials. Please make sure to review these prior labs and tutorials while completing this lab. 

# ## Environment

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import copy
import torch.nn.utils.prune as prune
from torchvision import datasets, models, transforms
from pathlib import Path
import os
import time
!rm data || rm -rf data
!mkdir data
print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)
import gc
torch.manual_seed(0)
np.random.seed(0)

rm: cannot remove 'data': No such file or directory
PyTorch Version:  1.12.1+cu113
Torchvision Version:  0.13.1+cu113


In [2]:
device_to_use = 'cuda'

In [3]:
# Mount data directory to google drive
from google.colab import drive
drive.mount('./drive')

# Copy to local for faster extraction
!cp drive/Shareddrives/penn-ese-5390-202230/ILSVRC2012_img_val.tar ./data
!cp drive/Shareddrives/penn-ese-5390-202230/ILSVRC2012_devkit_t3.tar.gz ./data
!cp drive/Shareddrives/penn-ese-5390-202230/ILSVRC2012_devkit_t12.tar.gz ./data

# drive.mount('./drive', force_remount=True)
# !cp /content/drive/Shareddrives/539/ILSVRC2012_img_val.tar ./data

# !cp /content/drive/Shareddrives/539/ILSVRC2012_devkit_t3.tar.gz ./data

# !cp /content/drive/Shareddrives/539/ILSVRC2012_devkit_t12.tar.gz ./data
print('Finished copying from Drive to local.')

Mounted at ./drive
Finished copying from Drive to local.


In [4]:
# Create transform to preprocess data
val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create validation dataset
val_dataset = datasets.ImageNet('./data', split='val', transform=val_transform)

# Create validation dataloader
val_dataloader = torch.utils.data.DataLoader(val_dataset, shuffle=False, num_workers=2)

print(f'Number of validation images: {len(val_dataset)}')

Number of validation images: 50000


In [5]:
# Put your reusable functions here.
# You can copy functions from previous labs and tutorials.
def validate_model(model, dataloader, n=1024, device = 'cpu'):
      model.eval()
      running_corrects = 0
      total = 0

      # Move model to device (CPU or GPU)
      # One time to amortize data movement
      dev = torch.device(device)
      model.to(dev)

      # Iterate over data stopping early if n is set
      t1 = time.time()
      for i, (inputs, labels) in enumerate(dataloader):
          if (n is not None and i >= n):
              break
          # Send inputs to device
          inputs = inputs.to(dev)

          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)
          
          """
          """
          temp = labels.data
          if(device=="cuda"):
            temp = temp.to(device="cuda")


          # Gather statistics
          running_corrects += torch.sum(predicted == temp)
          total += inputs.size()[0]       # get batch size

          if i % 200 == 199:
              acc = 100 * running_corrects.double() / total
              print(f'[{i + 1}] {acc:.4f}%')
      t2 = time.time()
      exe_time = t2-t1
      epoch_acc = 100 * running_corrects.double() / total
      del model
      del inputs
      gc.collect()
      
      torch.cuda.empty_cache()
      return epoch_acc, exe_time
def get_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    model_size = os.path.getsize("temp.p")/1e6
    os.remove('temp.p')
    return model_size

## Question 1: Global Pruning

- Perform pruning using `prune.global_unstructured` on a pretrained VGG16 using various sparsity targets.
- The models should be pruned away at 5% increments from 5% to 45% total pruning of the model.
- Save a copy of each pruned model in a dictionary.
- Plot the top-1 accuracy against weight pruned from 0% to 45% (use the original VGG16 for 0%).

In [6]:
orig_model = models.vgg16(pretrained = True)#TODO: load pretrained model
orig_model.eval()
orig_acc, orig_time = validate_model(orig_model, val_dataloader, 1024, device = device_to_use)
print(f'Validation Finished. Original Time: {orig_time:.1f}s. Original Accuracy: {orig_acc:.1f}%')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

[200] 85.5000%
[400] 82.0000%
[600] 84.8333%
[800] 87.0000%
[1000] 87.4000%
Validation Finished. Original Time: 22.3s. Original Accuracy: 87.3%


In [ ]:
prune_rate_list = range(5, 50, 5) # List of prune rate to test from 5-45 inclusive with step of 5
global_pruning = {} # Dictionary to store global pruning results
for prune_rate in prune_rate_list:
    model = orig_model #TODO: load pretrained model
    parameter_to_prune = (
        (model.features[0], 'weight'), # conv1 of VGG16
        (model.features[2], 'weight'),
        (model.features[5], 'weight'),
        (model.features[7], 'weight'),
        (model.features[10], 'weight'),
        (model.features[12], 'weight'),
        (model.features[14], 'weight'),
        (model.features[17], 'weight'),
        (model.features[19], 'weight'),
        (model.features[21], 'weight'),
        (model.features[24], 'weight'),
        (model.features[26], 'weight'),
        (model.features[28], 'weight'),
        (model.classifier[0], 'weight'),
        (model.classifier[3], 'weight'),
        (model.classifier[6], 'weight')
        #TODO: Add more layers to prune
    )
    #TODO: Prune model
    prune.global_unstructured(parameter_to_prune,pruning_method=prune.L1Unstructured,amount=prune_rate/100.0,)
    #TODO: make the pruning permanent to increase speed
    for param, name in parameter_to_prune:
      prune.remove(param, name)
    global_pruning[prune_rate] = {} # Dictionary to store accuracy results and model for each prune rate
    global_pruning[prune_rate]['model'] = model.state_dict() # Copy pruned model to dictionary
    # TODO Run validation on the pruned model
    pruned_acc, pruned_time = validate_model(model, val_dataloader, 1024, device = device_to_use)
    del model
    torch.cuda.clear_cache
    print(f'Validation Finished. Pruned Time: {pruned_time:.1f}s. Pruned Accuracy: {pruned_acc:.1f}%')
    print(f'Speedup: {orig_time/pruned_time}x. Accuracy drop: {orig_acc-pruned_acc}%')
    global_pruning[prune_rate]['top1_acc'] = pruned_acc # TODO fill with top1 accuracy
    global_pruning[prune_rate]['top1_acc_rel'] = pruned_acc/orig_acc*100 # Percent accuracy compared to original model
    print(f'Top1 accuracy for prune amount {prune_rate}%: {global_pruning[prune_rate]["top1_acc"]}%')
    print(f'Top1 accuracy (rel) for prune amount {prune_rate}%: {global_pruning[prune_rate]["top1_acc_rel"]}%')

# TODO plot the results

[200] 85.5000%
[400] 81.7500%
[600] 84.6667%
[800] 86.8750%
[1000] 87.3000%
Validation Finished. Pruned Time: 19.7s. Pruned Accuracy: 87.2%
Speedup: 1.0262224924891417x. Accuracy drop: 0.09765625%
Top1 accuracy for prune amount 5%: 87.20703125%
Top1 accuracy (rel) for prune amount 5%: 99.88814317673378%
[200] 85.5000%
[400] 82.0000%
[600] 84.8333%
[800] 87.1250%
[1000] 87.5000%
Validation Finished. Pruned Time: 22.9s. Pruned Accuracy: 87.4%
Speedup: 0.8825931540186771x. Accuracy drop: -0.09765625%
Top1 accuracy for prune amount 10%: 87.40234375%
Top1 accuracy (rel) for prune amount 10%: 100.11185682326622%
[200] 85.5000%
[400] 82.0000%
[600] 84.8333%
[800] 87.1250%
[1000] 87.5000%
Validation Finished. Pruned Time: 22.7s. Pruned Accuracy: 87.4%
Speedup: 0.8903181826007368x. Accuracy drop: -0.09765625%
Top1 accuracy for prune amount 15%: 87.40234375%
Top1 accuracy (rel) for prune amount 15%: 100.11185682326622%
[200] 85.5000%
[400] 82.0000%
[600] 84.8333%
[800] 87.1250%
[1000] 87.5000%
V

## Question 2: Layer-wise Pruning

- Perform pruning using `prune.l1_unstructured` on each layer of pretrained VGG16 using various sparsity targets.
- Each layer should be pruned away at 5% increments from 5% to 45% total pruning of the model.
- Save a copy of each pruned model in a dictionary.
- Plot the top-1 accuracy against weight pruned from 0% to 45% (use the original VGG16 for 0%).
- What do you observe? Why does layer-wise pruning perform better/worse than global pruning?

orig_model = #TODO: load pretrained model
prune_rate_list = range(5, 50, 5) # List of prune rate to test from 5-45 inclusive with step of 5
layer_pruning = {} # Dictionary to store layer pruning results
for prune_rate in prune_rate_list:
    model = #TODO: load pretrained model
    convs_to_prune = () #TODO: Add conv layers to prune
    linears_to_prune = () #TODO: Add linear layers to prune
    #TODO: Prune model
    layer_pruning[prune_rate_list] = {} # Dictionary to store accuracy results and model for each prune rate
    layer_pruning[prune_rate]['model'] = model # Copy pruned model to dictionary
    # TODO Run validation on the pruned model
    layer_pruning[prune_rate]['top1_acc'] = None # TODO fill with top1 accuracy
    layer_pruning[prune_rate]['top1_acc_rel'] = None # Percent accuracy compared to original model
    print(f'Top1 accuracy for prune amount {prune_rate}%: {layer_pruning[prune_rate]["top1_acc"]}%')
    print(f'Top1 accuracy (rel) for prune amount {prune_rate}%: {layer_pruning[prune_rate]["top1_acc_rel"]}%')

# TODO plot the results

In [ ]:
prune_rate_list = range(5, 50, 5) # List of prune rate to test from 5-45 inclusive with step of 5
layer_pruning = {} # Dictionary to store global pruning results
for prune_rate in prune_rate_list:
    model = orig_model #TODO: load pretrained model
    parameter_to_prune = (
        (model.features[0], 'weight'), # conv1 of VGG16
        (model.features[2], 'weight'),
        (model.features[5], 'weight'),
        (model.features[7], 'weight'),
        (model.features[10], 'weight'),
        (model.features[12], 'weight'),
        (model.features[14], 'weight'),
        (model.features[17], 'weight'),
        (model.features[19], 'weight'),
        (model.features[21], 'weight'),
        (model.features[24], 'weight'),
        (model.features[26], 'weight'),
        (model.features[28], 'weight'),
        (model.classifier[0], 'weight'),
        (model.classifier[3], 'weight'),
        (model.classifier[6], 'weight')
        #TODO: Add more layers to prune
    )
    #TODO: Prune model
    for param, name in parameter_to_prune:
      prune.l1_unstructured(param, name=name, amount=prune_rate/100.0)
    #TODO: make the pruning permanent to increase speed
    for param, name in parameter_to_prune:
      prune.remove(param, name)
    layer_pruning[prune_rate] = {} # Dictionary to store accuracy results and model for each prune rate
    layer_pruning[prune_rate]['model'] = model.state_dict() # Copy pruned model to dictionary
    #TODO Run validation on the pruned model
    pruned_acc, pruned_time = validate_model(model, val_dataloader, 1024, device = device_to_use)
    torch.cuda.clear_cache
    print(f'Validation Finished. Pruned Time: {pruned_time:.1f}s. Pruned Accuracy: {pruned_acc:.1f}%')
    print(f'Speedup: {orig_time/pruned_time}x. Accuracy drop: {orig_acc-pruned_acc}%')
    layer_pruning[prune_rate]['top1_acc'] = pruned_acc # TODO fill with top1 accuracy
    layer_pruning[prune_rate]['top1_acc_rel'] = pruned_acc/orig_acc*100 # Percent accuracy compared to original model
    print(f'Top1 accuracy for prune amount {prune_rate}%: {layer_pruning[prune_rate]["top1_acc"]}%')
    print(f'Top1 accuracy (rel) for prune amount {prune_rate}%: {layer_pruning[prune_rate]["top1_acc_rel"]}%')

# TODO plot the results

[200] 85.5000%
[400] 82.0000%
[600] 84.8333%
[800] 87.1250%
[1000] 87.5000%
Validation Finished. Pruned Time: 13.3s. Pruned Accuracy: 87.4%
Speedup: 1.4896675240135997x. Accuracy drop: -0.09765625%
Top1 accuracy for prune amount 5%: 87.40234375%
Top1 accuracy (rel) for prune amount 5%: 100.11185682326622%
[200] 86.0000%
[400] 82.2500%
[600] 85.0000%
[800] 87.2500%
[1000] 87.7000%
Validation Finished. Pruned Time: 13.4s. Pruned Accuracy: 87.6%
Speedup: 1.4844749690833683x. Accuracy drop: -0.29296875%
Top1 accuracy for prune amount 10%: 87.59765625%
Top1 accuracy (rel) for prune amount 10%: 100.33557046979867%
[200] 86.0000%
[400] 82.2500%
[600] 85.0000%
[800] 87.3750%
[1000] 87.6000%
Validation Finished. Pruned Time: 13.7s. Pruned Accuracy: 87.5%
Speedup: 1.4479009763522686x. Accuracy drop: -0.1953125%
Top1 accuracy for prune amount 15%: 87.5%
Top1 accuracy (rel) for prune amount 15%: 100.22371364653245%
[200] 86.5000%
[400] 82.2500%
[600] 85.1667%
[800] 87.5000%
[1000] 87.8000%
Validat

## Question 3: Layer-wise (Input) Channel Pruning

- Perform pruning using `prune.ln_structured` on each layer of pretrained VGG16 using various sparsity targets.
- Prune along the input channels for conv layers and input dimension for linear layers
- Do not prune the first conv layer
- Each layer should be pruned away at 5% increments from 5% to 45% total pruning of the model.
- Save a copy of each pruned model in a dictionary.
- Plot the top-1 accuracy against weight pruned from 0% to 45% (use the original VGG16 for 0%).
- What do you observe? Why does layer-wise channel pruning perform better/worse than layer-wise unstructured pruning?

In [ ]:
# prune_rate_list = range(5, 50, 5) # List of prune rate to test from 5-45 inclusive with step of 5
# channel_pruning = {} # Dictionary to store channel pruning results
# for prune_rate in prune_rate_list:
#     model = orig_model#TODO: load pretrained model
#     convs_to_prune = () #TODO: Add conv layers to prune (except the first conv layer)
#     linears_to_prune = () #TODO: Add linear layers to prune
#     #TODO: Prune model
#     channel_pruning[prune_rate] = {} # Dictionary to store accuracy results and model for each prune rate
#     channel_pruning[prune_rate]['model'] = model # Copy pruned model to dictionary
#     # TODO Run validation on the pruned model
#     channel_pruning[prune_rate]['top1_acc'] = None # TODO fill with top1 accuracy
#     channel_pruning[prune_rate]['top1_acc_rel'] = None # Percent accuracy compared to original model
#     print(f'Top1 accuracy for prune amount {prune_rate}%: {channel_pruning[prune_rate]["top1_acc"]}%')
#     print(f'Top1 accuracy (rel) for prune amount {prune_rate}%: {channel_pruning[prune_rate]["top1_acc_rel"]}%')

# TODO plot the results

prune_rate_list = range(5, 50, 5) # List of prune rate to test from 5-45 inclusive with step of 5
channel_pruning = {} # Dictionary to store global pruning results
for prune_rate in prune_rate_list:
    model = orig_model #TODO: load pretrained model

    convs_to_prune = (
        (model.features[0], 'weight'), # conv1 of VGG16
        (model.features[2], 'weight'),
        (model.features[5], 'weight'),
        (model.features[7], 'weight'),
        (model.features[10], 'weight'),
        (model.features[12], 'weight'),
        (model.features[14], 'weight'),
        (model.features[17], 'weight'),
        (model.features[19], 'weight'),
        (model.features[21], 'weight'),
        (model.features[24], 'weight'),
        (model.features[26], 'weight'),
        (model.features[28], 'weight'),
    )
    linears_to_prune = (
        (model.classifier[0], 'weight'),
        (model.classifier[3], 'weight'),
        (model.classifier[6], 'weight'),
    )
    #TODO: Prune model
    for param, name in convs_to_prune:
        prune.ln_structured(param, name=name, amount=prune_rate/100.0, n=2, dim=0)
    for param, name in linears_to_prune:
        prune.ln_structured(param, name=name, amount=prune_rate/100.0, n=2, dim=1)
    #TODO: make the pruning permanent to increase speed
    for param, name in convs_to_prune:
        prune.remove(param, name)
    for param, name in linears_to_prune:
        prune.remove(param, name)
    channel_pruning[prune_rate] = {} # Dictionary to store accuracy results and model for each prune rate
    channel_pruning[prune_rate]['model'] = model.state_dict() # Copy pruned model to dictionary
    #TODO Run validation on the pruned model
    pruned_acc, pruned_time = validate_model(model, val_dataloader, 1024, device = device_to_use)
    print(f'Validation Finished. Pruned Time: {pruned_time:.1f}s. Pruned Accuracy: {pruned_acc:.1f}%')
    print(f'Speedup: {orig_time/pruned_time}x. Accuracy drop: {orig_acc-pruned_acc}%')
    channel_pruning[prune_rate]['top1_acc'] = pruned_acc # TODO fill with top1 accuracy
    channel_pruning[prune_rate]['top1_acc_rel'] = pruned_acc/orig_acc*100 # Percent accuracy compared to original model
    print(f'Top1 accuracy for prune amount {prune_rate}%: {channel_pruning[prune_rate]["top1_acc"]}%')
    print(f'Top1 accuracy (rel) for prune amount {prune_rate}%: {channel_pruning[prune_rate]["top1_acc_rel"]}%')

# TODO plot the results


[200] 85.0000%
[400] 77.2500%
[600] 78.8333%
[800] 81.3750%
[1000] 80.8000%
Validation Finished. Pruned Time: 12.6s. Pruned Accuracy: 80.9%
Speedup: 1.4980694757066781x. Accuracy drop: 6.4453125%
Top1 accuracy for prune amount 5%: 80.859375%
Top1 accuracy (rel) for prune amount 5%: 92.61744966442953%
[200] 80.0000%
[400] 66.0000%
[600] 65.3333%
[800] 71.0000%
[1000] 71.0000%
Validation Finished. Pruned Time: 12.7s. Pruned Accuracy: 71.0%
Speedup: 1.4889027007959605x. Accuracy drop: 16.30859375%
Top1 accuracy for prune amount 10%: 70.99609375%
Top1 accuracy (rel) for prune amount 10%: 81.31991051454138%
[200] 54.0000%
[400] 41.0000%
[600] 38.6667%
[800] 42.8750%
[1000] 43.4000%
Validation Finished. Pruned Time: 17.4s. Pruned Accuracy: 43.6%
Speedup: 1.0816037757057129x. Accuracy drop: 43.75%
Top1 accuracy for prune amount 15%: 43.5546875%
Top1 accuracy (rel) for prune amount 15%: 49.88814317673378%
[200] 28.5000%
[400] 18.7500%
[600] 17.6667%
[800] 20.0000%
[1000] 17.9000%
Validation Fi

## Question 4: Layer-wise (Input) Channel Pruning (Continued)

In this question, we harden the channel pruning by removing the channels
- Collect the number of non-zero input channels for each layer
- Instantiate a new model based on the number of non-zero input channels
- Remember that the output channels of previous layer should match the input channels for current layer
- Pay special attention to the last convolution layer and first linear layer
- Copy over the the non-zero weights from the pruned model to the hardened model
- Calculate the run time and model size for the original VGG16 and the hardened VGG16s
- Plot the **relative** top-1 accuracy, run time and model size from 0% to 45% pruned VGG16
- What do you observe? What is the trade-off between accuracy, run time, and model size?

In [ ]:
prune_rate_list = range(5, 50, 5) # List of prune rate to test from 5-45 inclusive with step of 5
hardened_pruning = {} # Dictionary to store hardened pruning results
for prune_rate in prune_rate_list:
    #TODO: Collect the number of non-zero channels for each layer
    #TODO: Instantiate a new model based on collected number of non-zero channels
    hardened_pruning[prune_rate_list] = {} # Dictionary to store accuracy results and model for each prune rate
    hardened_pruning[prune_rate]['model'] = model # Copy original model to dictionary
    hardened_pruning[prune_rate]['top1_acc'] = None # TODO just copy from channel pruning
    hardened_pruning[prune_rate]['top1_acc_rel'] = None # TODO just copy from channel pruning
    hardened_pruning[prune_rate]['run_time'] = None # TODO Collect run time of the hardened model
    hardened_pruning[prune_rate]['run_time_rel'] = None # TODO Collect run time (relative) of the hardened model
    hardened_pruning[prune_rate]['model_size'] = None # TODO Collect model size of the hardened model
    hardened_pruning[prune_rate]['model_size_rel'] = None # TODO Collect model size (relative) of the hardened model

In [ ]:
#TODO plot the relative accuracy, relative run time, and relative model size

## (Optional) Question 5: Regularization to promote sparsity
Using lab 1 as guidance, load a pretrained trained VGG16 model without batch normalization (without `_bn` suffix) below and train it using Tiny ImageNet. The specifications are listed below:

* The data transform should be according to VGG16_weights parameter description found [here](https://pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html).
* Use a batch size of `32`
* Use the default `CrossEntropyLoss` loss function as a criterion
* Use stochastic gradient descent with learning rate of `1e-3`, momentum of `0.9`, weight decay of `1e-4`
* Use a momentum value of `0.9`
* Use a weight decay of `1e-4`
* Train for 10 Epochs
* Train the model with GPU
* save the trained model in a file called `original_net`

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
  
# Unzip raw zip file
!unzip -qq 'tiny-imagenet-200.zip'

# Define main data directory
data_dir = Path('tiny-imagenet-200') # Original images come in shapes of [3,64,64]

# Define training and validation data paths
train_dir = data_dir / 'train'
valid_dir = data_dir / 'val'

In [ ]:
from torch.utils.data.datapipes.datapipe import T
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
trainset = datasets.ImageFolder(train_dir, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)  # Batch size of 32
criterion = nn.CrossEntropyLoss() # Store loss function in this variable
original_net = models.efficientnet_b0()
print(original_net)

In [ ]:
optimizer = torch.optim.SGD(original_net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4) # Store SGD optimizer in this variable

# TODO write reuseable validation and training functions here to reduce work
# TODO fill out the above variables
# TODO run training
# TODO save the trained model to a file called original_net

## (Optional) Question 6: Accuracy on TinyImageNet

Run validation over the entire validation set and report the top 1 and top 5 accuracy.

In [ ]:
validset = datasets.ImageFolder(valid_dir, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, num_workers=2)
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in validloader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = original_net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

## (Optional) Question 7: Iterative Pruning
Follow the iterative pruning method described in lecture co-design II. Run three iterations of iterative pruning--training for an entire epoch each iteration. The final target prune rate follows the following equation $final = 1 - (1 - r)^n$. Therefore, ensure to pick the correct iterative prune rate, $r$, to meet the final prune rate target $final$. The final prune rate targets are captured in the `prune_rate_list`. $n$ is the number of iterations.

- Compare your accuracies to Q1-4.

In [ ]:
iterative_pruning = {} # Dictionary to store pruned model after retraining and accuracy for each prune rate target
iterative_prune_rate = None # iterative prune rate needed to achieve `prune_rate` final target when pruned three times
for prune_rate in prune_rate_list:
    iterative_pruning[prune_rate] = {} # Dictionary to store accuracy and model for each sparsity value
    iterative_pruning[prune_rate]['model'] = copy.deepcopy(original_net) # Copy original model to dictionary
    # TODO Run iterative pruning three times
    iterative_pruning[prune_rate]['top1_acc'] = None # TODO fill with top1 accuracy
    iterative_pruning[prune_rate]['top1_acc_loss'] = None # Percent accuracy loss compared to original model
    print(f'Top1 accuracy for sparsity {prune_rate}%: {iterative_pruning[prune_rate]["top1_acc"]}%')
    print(f'Top1 accuracy loss for sparsity {prune_rate}%: {iterative_pruning[prune_rate]["top1_acc_loss"]}%')
# TODO save the trained model to a file called iterative_pruned_net